Цель: В этом домашнем задании вам предстоит обойти все ловушки серверов, пробраться сквозь страницы html-кода и собрать себе свой собственный датасет.

По аналогии с занятием, возьмите интересующий вас сайт, на котором можно пособирать какие-то данные (и при этом API не предоставляется).
Напишите свой парсер, который будет бегать по страничкам и автоматически что-то собирать.

В качестве потенциальных целей для парсинга можно взять какие-нибудь блоги, выкачать оттуда публикации, авторов, число просмотром и комментариев. Можно посмотреть новостные ресурсы и скачать новостные статьи с их мета-информацией. Или любой другой интересный вам сайт.

Не забывайте, что парсинг - это ответственное мероприятие, поэтому не бомбардируйте несчастные сайты слишком частыми запросами (можно ограничить число запросов в секунду при помощи time.sleep(0.3), вставленного в теле цикла)

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys

from tqdm import tqdm_notebook

In [2]:
news_url = 'http://ratel.kz/raw/?page='

In [3]:
home_url = 'http://ratel.kz'

In [4]:
page = 1

In [5]:
requests.get(news_url+str(page))

<Response [200]>

In [6]:
news_page = requests.get(news_url+str(page))
news_page = news_page.content

In [7]:
news_page = BeautifulSoup(news_page, 'lxml')

In [8]:
news_page = news_page.find('div', attrs={'class':'col-sm-8 col-sm-push-1'})

In [74]:
def get_articles(news_page):
    articles = news_page.findAll('div', attrs={'class':'article article_list'})
    return articles

In [75]:
articles = get_articles(news_page)

In [60]:
def get_article(news_page):
    article = news_page.find('div', attrs={'class':'article article_list'})
    return article

#### Заголовки новостей

In [35]:
def get_header(article):
    header = article.find('div', attrs = {'class':'article__title'})
    header = header.text.strip()
    return header

In [36]:
get_header(news_page)

'Сообщения о задержании детей не подтвердились'

In [76]:
for article in articles:
    print(get_header(article))

Сообщения о задержании детей не подтвердились
Нур-Султан: за шесть часов до карантина
22 марта Алматы полностью закроют
Волонтёры сейчас нужнее дорог
Магазины, кафе и салоны красоты в условиях ЧП должны работать с 10 утра до 18:00
В Усть-Каменогорске зарегистрировали первые случаи мошенничества с медицинскими масками
Обстановка нервная, но паники нет
Цены не слушаются акимата Петропавловска
В Талдыкоргане от взрыва газа пострадала беременная женщина
Заместитель Генпрокурора обратился к гражданам
Коронавирус шыққан елдің жүк көлігінің жүргізушісі Қазақстан аумағына жіберілмейді
Слухи о пациентах  с COVID-19 распространяла директор школы города Текели
Казахстанцев, отдыхающих по турпутевкам за границей, начали возвращать на родину
Число заражённых в Алматы составляет 15 человек
Кто сможет въехать и выехать из Алматы и Нур-Султана во время карантина
Родные погибшей: Мы не "ворошиловские стрелки"


#### Короткий текст статьи

In [40]:
def get_short_text(article):
    short_text = article.find('div', attrs = {'class':'article__text'}).text
    short_text = short_text.strip()
    return short_text

In [42]:
for article in articles:
    print(get_short_text(article))

Полиция Алматинской области устанавливает распространителей ложной информации
Наш корреспондент прошлась по столице и увидела почти обезлюдевший город
72 часа дали алматинцам на то, чтобы вернуться домой из других областей
В Павлодарской области решено за счёт сокращения объемов ремонта дорог поощрить учителей, тренеров и волонтёров
Рекомендации опубликованы в соцсетях
Предприимчивые граждане рекламируют услуги по продаже и доставке востребованной продукции на сайтах и пропадают после оплаты
Карагандинцы спешат сдать или обменять билеты на поезда в связи с режимом ЧП и карантином
Власти Петропавловска говорят, что цены на продукты не повышаются, цены говорят обратное
С ожогами первой степени её госпитализировали в больницу
Казахстанцам рекомендовано проходить лечение и обследование, иначе - административная или уголовная ответственность
Қазақстанның шекара қызметі COVID-19 пандемиясына қарсы қосымша шаралар қолданады
Педагог уже принесла публичные извинения
Первый борт был отправлен во

#### Article page

In [77]:
def get_link(news_page):
    link = news_page.find('a', attrs= {'class':'article__img-link'}).get('href')
    return link

In [81]:
news_link = get_link(news_page)
news_link

'/raw/soobschenija_o_zaderzhanii_detej_ne_podtverdilis'

#### Страничка одной статьи

In [14]:
def get_subpage(link):
    sub_page = requests.get(home_url+str(link)).content
    sub_page = BeautifulSoup(sub_page, 'lxml')
    return sub_page

In [15]:
sub_page = get_subpage(news_link)

#### Просмотры статьи

In [16]:
def get_views(sub_page):
    post_views = sub_page.find('div', attrs={'class':'post__views'})
    text = post_views.text
    views = text.strip().split()[0]
    return int(views)

get_views(sub_page)

315

#### Дата публикации

In [17]:
import datetime
import locale
locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [18]:
def get_date(news_page):
    pattern = re.compile(r'[\d]{1,2}[\s]{1}[А-я]{3}[\s]{1}[\d]{4}')
    text = news_page.find('div', attrs = {'class':'article__date'}).text
    date = re.match(pattern, text).group(0)
    return date if date else ""

get_date(news_page)

'18 Мар 2020'

In [53]:
def get_datetime(article):
    article_info = article.find('div', attrs = {'class':'article__date'}).text
    time_data = article_info.split(" — ")[0]
    date = datetime.datetime.strptime(time_data, '%d %b %Y, %H:%M')
    return date

In [54]:
for article in articles:
    print(get_datetime(article))

2020-03-18 19:00:00
2020-03-18 18:00:00
2020-03-18 17:30:00
2020-03-18 17:00:00
2020-03-18 16:40:00
2020-03-18 14:40:00
2020-03-18 13:50:00
2020-03-18 13:00:00
2020-03-18 12:30:00
2020-03-18 11:20:00
2020-03-18 10:20:00
2020-03-18 09:40:00
2020-03-18 08:30:00
2020-03-18 08:00:00
2020-03-17 22:25:00
2020-03-17 19:40:00


https://stackoverflow.com/questions/41839678/how-to-use-russian-date-string-with-strptime


#### Автор статьм

In [21]:
article = news_page.find('div', attrs= {'class':'article__date'})

In [45]:
def get_author(article):
    article_info = article.find('div', attrs = {'class':'article__date'}).text
    article_info = article_info.split(" — ")
    author = article_info[1].strip()
    return author

In [46]:
for article in articles:
    print(get_author(article))

Алия АХМЕДИЕВА
Асем  МИРЖЕКЕЕВА
Ратель
Ольга ВОРОНЬКО
Ратель
Антон СЕРГИЕНКО
Сергей ПЕРХАЛЬСКИЙ
Ольга ВАЙТОВИЧ
Алия АХМЕДИЕВА
Ратель
Ратель
Алия АХМЕДИЕВА
Ратель
Ратель
Ратель
Ольга ВОРОНЬКО


In [55]:
news_link

'/raw/soobschenija_o_zaderzhanii_detej_ne_podtverdilis'

In [86]:
def get_article_data(news_page):
    news_link = get_link(news_page)
    sub_page = get_subpage(news_link)
    article = get_article(news_page)
    article_data = {
        'views': get_views(sub_page),
        'header': get_header(article),
        'author': get_author(article),
        'date': get_date(article),
        'short_text': get_short_text(article)
    }
    return article_data

In [88]:
def get_news(page):
    news_page = requests.get(news_url+str(page))
    news_page = news_page.content
    news_page = BeautifulSoup(news_page, 'lxml')
    
    news_page = news_page.find('div', attrs={'class':'col-sm-8 col-sm-push-1'})
    article_data = get_article_data(news_page)
    
    return article_data

In [89]:
news = get_news(1)

In [90]:
news

{'author': 'Алия АХМЕДИЕВА',
 'date': '18 Мар 2020',
 'header': 'Сообщения о задержании детей не подтвердились',
 'short_text': 'Полиция Алматинской области устанавливает распространителей ложной информации',
 'views': 401}

## Parsing start

In [ ]:
# for news_link in news_links:
#     sub_page_data = get_sub_page_data(news_link)
#     print(sub_page_data['header'])
#     print(sub_page_data['date'])
#     print(f"Likes: {sub_page_data['likes']}, Dislikes: {sub_page_data['dislikes']}")
#     print('====================')
#     time.sleep(0.3)